In [ ]:
# Install openpyxl to open excel file
# !pip install openpyxl
# Install spacy for nlp task
#!pip install spacy
#!pip install pyLDAvis
#!python -m pip install -U gensim
#!pip install nltk

# Topic modeling of _De avond is ongemak_ by Marieke Lucas Rijneveld

In this notebook, part 3 (Deel III) of Marieke Lucas Rijneveld's novel _De avond is ongemak_ is analyzed using topic modeling. This notebook is part of a three-part analysis of the novel, which was created in the course of the master thesis ''Naar de overkant: Motieven en topics in Marieke Lucas Rijnevelds _De avond is ongemak_'', written in 2021-2022 at the University of Vienna (degree program: Nederlandistik). All comments are written in English. 

## 1. Create dataframe

### 1.1. Load required libraries and packages

In [2]:
# Load the required libraries and packages
import os # Supports interaction with the operating system
import pandas as pd # Package to manipulate data structures
import re # Enables working with regular expressions

### 1.2. Convert text file into a data frame
In a first step the raw text file is converted into a data frame, a two-dimensional table format, for further processing and subsequent analysis.

In [3]:
# Splits .txt file into chunks (sentences) and saves them row after row in a pandas dataframe
string = [] 

with open("data/texts/De_avond_is_ongemak_Deel_III.txt", "r", encoding='utf8') as f:
    full_text = f.read()
    for l in re.split(r"(\.)", full_text):
        if l != ".":
            string.append(l + "\n")
df = pd.DataFrame(string)

In [4]:
# .head() outputs the first 5 rows of the created data frame called "df"
df.head()

,0
0,DEEL III\n\n\n\n\n\n\t\t \t\t\t-\n\n\n\n\n\n1\...
1,Door de opening tussen de gordijnen valt een ...
2,De rode snede van het bonken is een litteken ...
3,Ik knijp mijn ogen stijf dicht en voel zijn w...
4,Gelukkig zijn het alleen mijn oorschelpen en ...


In [5]:
# Assign column name "text"
df.columns =['text']
df.head()

,text
0,DEEL III\n\n\n\n\n\n\t\t \t\t\t-\n\n\n\n\n\n1\...
1,Door de opening tussen de gordijnen valt een ...
2,De rode snede van het bonken is een litteken ...
3,Ik knijp mijn ogen stijf dicht en voel zijn w...
4,Gelukkig zijn het alleen mijn oorschelpen en ...


In [6]:
# Save data frame as .csv file for intermediate manual verification steps
df.to_csv('data/DAIO_Deel_III_sentences.csv', encoding='utf8', sep=';')

## 2. Text preprocessing and tokenization

### 2.1. Load the required libraries and packages

In [7]:
# Import libraries and packages
import re # for regular expressions
import gensim # for topic modeling
from gensim.parsing.preprocessing import STOPWORDS # package for stop word processing

import nltk # natural language toolkit
from nltk.tag import PerceptronTagger # part-of-speech tagging package
from nltk.corpus import alpino as alp # dutch corpus for PerceptronTagger
nltk.download('alpino') # download dutch corpus
from nltk.stem import WordNetLemmatizer # for lemmatization
from nltk.corpus import stopwords # package for stop word processing
from nltk.stem.snowball import DutchStemmer # for stemming

import numpy as np # for working with numerical data

import codecs # codec module

import simplemma # for lemmatization

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### 2.2. Create stop word list
A stop word list contains words wich are unnecessary/responsible for negative influence on the generation of topics.

In [9]:
# Open stopword list .txt file for Dutch stopwords
stopword_file = open('data/stopwords_nl.txt', 'r')

In [10]:
# Save stop word file as variable
stopwords = stopword_file.read()

In [11]:
# Print stopword list
print(stopwords)

aan,
aangaande,
aangezien,
achte,
achter,
achterna,
af,
afgelopen,
al,
aldaar,
aldus,
alhoewel,
alias,
alle,
allebei,
alleen,
alles,
als,
alsnog,
altijd,
altoos,
ander,
andere,
anders,
anderszins,
beetje,
behalve,
behoudens,
beide,
beiden,
ben,
beneden,
bent,
bepaald,
betreffende,
bij,
bijna,
bijv,
binnen,
binnenin,
blijkbaar,
blijken,
boven,
bovenal,
bovendien,
bovengenoemd,
bovenstaand,
bovenvermeld,
buiten,
bv,
daar,
daardoor,
daarheen,
daarin,
daarna,
daarnet,
daarom,
daarop,
daaruit,
daarvanlangs,
dan,
dat,
de,
deden,
deed,
der,
derde,
derhalve,
dertig,
deze,
dhr,
die,
dikwijls,
dit,
doch,
doe,
doen,
doet,
door,
doorgaand,
drie,
duizend,
dus,
echter,
een,
eens,
eer,
eerdat,
eerder,
eerlang,
eerst,
eerste,
eigen,
eigenlijk,
elk,
elke,
en,
enig,
enige,
enigszins,
enkel,
er,
erdoor,
erg,
ergens,
etc,
etcetera,
even,
eveneens,
evenwel,
gauw,
ge,
gedurende,
geen,
gehad,
gekund,
geleden,
gelijk,
gemoeten,
gemogen,
genoeg,
geweest,
gewoon,
gewoonweg,
haar,
haarzelf,
had,
hadden,
hare,
he

In [12]:
# Split stop word string into a list
stop_words = stopwords.split(',\n')

In [13]:
# Close stopword file
stopword_file.close()

In [14]:
# Display list with stopwords for words wich are unnecessary/responsible for negative influence on the generation of topics
stop_words

['aan',
 'aangaande',
 'aangezien',
 'achte',
 'achter',
 'achterna',
 'af',
 'afgelopen',
 'al',
 'aldaar',
 'aldus',
 'alhoewel',
 'alias',
 'alle',
 'allebei',
 'alleen',
 'alles',
 'als',
 'alsnog',
 'altijd',
 'altoos',
 'ander',
 'andere',
 'anders',
 'anderszins',
 'beetje',
 'behalve',
 'behoudens',
 'beide',
 'beiden',
 'ben',
 'beneden',
 'bent',
 'bepaald',
 'betreffende',
 'bij',
 'bijna',
 'bijv',
 'binnen',
 'binnenin',
 'blijkbaar',
 'blijken',
 'boven',
 'bovenal',
 'bovendien',
 'bovengenoemd',
 'bovenstaand',
 'bovenvermeld',
 'buiten',
 'bv',
 'daar',
 'daardoor',
 'daarheen',
 'daarin',
 'daarna',
 'daarnet',
 'daarom',
 'daarop',
 'daaruit',
 'daarvanlangs',
 'dan',
 'dat',
 'de',
 'deden',
 'deed',
 'der',
 'derde',
 'derhalve',
 'dertig',
 'deze',
 'dhr',
 'die',
 'dikwijls',
 'dit',
 'doch',
 'doe',
 'doen',
 'doet',
 'door',
 'doorgaand',
 'drie',
 'duizend',
 'dus',
 'echter',
 'een',
 'eens',
 'eer',
 'eerdat',
 'eerder',
 'eerlang',
 'eerst',
 'eerste',
 'ei

### 2.2. Stop word removing, lemmatization and tokenization processes

In [16]:
# Loads the fitting lemmatization data for the dutch language
langdata = simplemma.load_data('nl')

In [17]:
# Creates a function to lemmatize a token
def lemmatize(token):
    """Returns lemmatization of a token"""
    return simplemma.lemmatize(token, langdata)

In [18]:
# Creates a function to remove stopwords, tokenize and lemmatize the given text
def tokenize(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

### 2.3. Part-of-speech tagging

In [20]:
# Train the PoS tagger on the dutch "alp" corpus
training_corpus = list(alp.tagged_sents()) 
tagger = PerceptronTagger(load=True)
tagger.train(training_corpus)

In [21]:
# Display classes from the tagger function for possible exclusion of word groups
sorted(tagger.classes)

['#',
 '$',
 "''",
 '(',
 ')',
 ',',
 '.',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 '``',
 'adj',
 'adv',
 'comp',
 'comparative',
 'det',
 'fixed',
 'name',
 'noun',
 'num',
 'part',
 'pp',
 'prep',
 'pron',
 'punct',
 'tag',
 'verb',
 'vg']

### 2.4. Merging of the created processes

In [ ]:
# Variable 'punctuation' with special characters
punctuation = '!”$%&\’()*+,-./:;<=>?[\\]^_`{|}~•@'

In [22]:
# Combines the previously functions into one process
# apply the following: lowercase, strip punctuation, extra spaces or numbers in text

def preprocess_text(text):
    text = text.lower()  # lower case
    text = re.sub('[' + punctuation + ']+', ' ', text)  # strip punctuation
    text = re.sub('\s+', ' ', text)  # remove double spacing
    text = re.sub('([0-9]+)', '', text)  # remove numbers
    text_token_list = tokenize(text)  # apply lemmatization and tokenization
    pos_comment = tagger.tag(text_token_list) #apply PoS tagging
    text_pos=[word[0] for word in pos_comment if word[1] in ['noun', 'adj']] # remove every class but nouns and adjectives
    text = ' '.join([w for w in text_pos]) # add remaining words
    return text

In [23]:
# Function to read in, clean, preprocess and tokenize text in dataframe column and save tokens in a new column
def tokenize_text(df):
    df['tokens'] = df.text.apply(preprocess_text) # creates new column and saves the functions output into
    num_texts = len(df)
    print('Complete. Number of text chunks that have been cleaned and tokenized: {}'.format(num_texts))
    return df

In [24]:
# Remove string control characters [\\t, \\n, \\r, \t, \n, \r] from data frame strings
df = df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=False)
df.head()

,text
0,DEEL III -1Ineens duwt Obbe zijn mond tegen mi...
1,Door de opening tussen de gordijnen valt een ...
2,De rode snede van het bonken is een litteken ...
3,Ik knijp mijn ogen stijf dicht en voel zijn w...
4,Gelukkig zijn het alleen mijn oorschelpen en ...


### 2.5. Application of all functions on the data frame

In [25]:
# Apply tokenize_text on data frame and display cleaned and tokenized number of text chunks
texts_df  = tokenize_text(df)

Complete. Number of text chunks that have been cleaned and tokenized: 1343


In [26]:
# .head() outputs the first 5 rows of the created data frame called "texts_df"
texts_df.head()

,text,tokens
0,DEEL III -1Ineens duwt Obbe zijn mond tegen mi...,duwen obbe mond fluisteren langzaam nadrukkeli...
1,Door de opening tussen de gordijnen valt een ...,gordijn lichtstreep
2,De rode snede van het bonken is een litteken ...,rode snijden naad sok obbe obstakel fijn
3,Ik knijp mijn ogen stijf dicht en voel zijn w...,stijf dichten warm tandpasta woord kolk tromme...
4,Gelukkig zijn het alleen mijn oorschelpen en ...,gelukkig oorschelp moeder vader allerergst woo...


In [28]:
# Save new dataframe as .csv file
texts_df.to_csv('data/DAIO_Deel_III_preprocessed_df_all_noun_adj.csv', index=False, header=True, sep=';', encoding='utf-8')

## 3. Topic modeling

In [29]:
# Import libraries and packages
import spacy # NLP library

import gensim.corpora as corpora # for creating a corpora
from gensim.models import CoherenceModel # for creating the coherence model 

from pprint import pprint # provides capability to "pretty-print" data structures

import pyLDAvis.gensim_models # pyLDAvis gensim models
import pickle # for serializing and de-serializing a object structure
import pyLDAvis # data visualization tool

### 3.1. Preprocess tokens

In [30]:
# Assign new variable to data frame
preprocessed_texts = texts_df

In [31]:
# .head() outputs the first 5 rows of the created data frame called "preprocessed_texts"
preprocessed_texts.head()

,text,tokens
0,DEEL III -1Ineens duwt Obbe zijn mond tegen mi...,duwen obbe mond fluisteren langzaam nadrukkeli...
1,Door de opening tussen de gordijnen valt een ...,gordijn lichtstreep
2,De rode snede van het bonken is een litteken ...,rode snijden naad sok obbe obstakel fijn
3,Ik knijp mijn ogen stijf dicht en voel zijn w...,stijf dichten warm tandpasta woord kolk tromme...
4,Gelukkig zijn het alleen mijn oorschelpen en ...,gelukkig oorschelp moeder vader allerergst woo...


In [32]:
# Display info of preprocessed df
preprocessed_texts.info

<bound method DataFrame.info of                                                    text  \
0     DEEL III -1Ineens duwt Obbe zijn mond tegen mi...   
1      Door de opening tussen de gordijnen valt een ...   
2      De rode snede van het bonken is een litteken ...   
3      Ik knijp mijn ogen stijf dicht en voel zijn w...   
4      Gelukkig zijn het alleen mijn oorschelpen en ...   
...                                                 ...   
1338   Zonder er verder nog over na te denken, schop...   
1339  ’ Een harde klap volgt, het vriezerlichtje flo...   
1340                      Het wordt pikkedonker en stil   
1341                                         IJzig stil   
1342                                                      

                                                 tokens  
0     duwen obbe mond fluisteren langzaam nadrukkeli...  
1                                   gordijn lichtstreep  
2              rode snijden naad sok obbe obstakel fijn  
3     stijf dichten warm ta

In [33]:
# Create a object which contains the tokens of the preprocessed_texts data frame
texts = preprocessed_texts['tokens']

In [34]:
# .head() outputs the first 5 rows of the object called "texts"
texts.head()

0    duwen obbe mond fluisteren langzaam nadrukkeli...
1                                  gordijn lichtstreep
2             rode snijden naad sok obbe obstakel fijn
3    stijf dichten warm tandpasta woord kolk tromme...
4    gelukkig oorschelp moeder vader allerergst woo...
Name: tokens, dtype: object

In [35]:
# Save texts object values to list
data = texts.values.tolist()

In [36]:
# Create function to convert data into a list of tokens
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))

In [37]:
# Create list of lists
data_words = list(sent_to_words(data))

In [38]:
# Output data_words
data_words

[['duwen',
  'obbe',
  'mond',
  'fluisteren',
  'langzaam',
  'nadrukkelijk',
  'godin',
  'ver'],
 ['gordijn', 'lichtstreep'],
 ['rode', 'snijden', 'naad', 'sok', 'obbe', 'obstakel', 'fijn'],
 ['stijf', 'dichten', 'warm', 'tandpasta', 'woord', 'kolk', 'trommelvlies'],
 ['gelukkig',
  'oorschelp',
  'moeder',
  'vader',
  'allerergst',
  'woord',
  'gedenken',
  'boerderij'],
 ['verdrietig', 'godin', 'huis', 'naam', 'ijdel'],
 ['vaak', 'woord', 'ineenkrimpen', 'laak'],
 ['sims'],
 ['obbe', 'hangen', 'half'],
 ['kus'],
 ['keer'],
 ['poppetje', 'hoeve', 'werk', 'stinkend', 'rijk', 'val'],
 ['vraag', 'godverdomme', 'vader', 'aftershave', 'mix', 'kaneel'],
 ['moeder',
  'tevreden',
  'stem',
  'buik',
  'band',
  'pyjamabroek',
  'onderbroek',
  'bil'],
 ['obbe', 'mond', 'poepgat', 'duwen'],
 ['moeder', 'dag', 'poep'],
 ['doorgang', 'weleens', 'gang', 'gevang', 'mier', 'aquarium', 'zand'],
 ['obbe',
  'mouw',
  'shirt',
  'duwen',
  'bil',
  'voorzichtig',
  'zuinig',
  'wijsvinger',
  'd

### 3.2. Dictionary and corpus for the LDA Model

In [40]:
# Creates dictionary which contains the number of times a word appears in "data_words"
id2word = corpora.Dictionary(data_words)

count = 0
for k, v in id2word.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 duwen
1 fluisteren
2 godin
3 langzaam
4 mond
5 nadrukkelijk
6 obbe
7 ver
8 gordijn
9 lichtstreep
10 fijn


In [41]:
# Filter out tokens that appear in less than 5 documents, more than 0.5 documents, and only the top 10000 tokens
id2word.filter_extremes(no_below=5, no_above=0.5, keep_n=10000)

In [42]:
# Create corpus
texts = data_words

In [43]:
# Convert document into bag-of-words format
corpus = [id2word.doc2bow(text) for text in texts]

### 3.3. Create LDA model

In [44]:
# Variable for number of topics
num_topics = 5

In [45]:
# Build LDA model with pre defined corpus and dictionary
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)# Print the Keyword in the topics

In [46]:
# Print the most significant topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.109*"moeder" + 0.042*"elkaar" + 0.039*"hanna" + 0.029*"vader" + '
  '0.029*"mens" + 0.020*"plek" + 0.018*"dag" + 0.018*"stoel" + 0.016*"keren" + '
  '0.016*"overkant"'),
 (1,
  '0.069*"goed" + 0.034*"doden" + 0.027*"donker" + 0.024*"één" + 0.023*"bang" '
  '+ 0.023*"hard" + 0.020*"echt" + 0.020*"rustig" + 0.020*"pad" + '
  '0.020*"snel"'),
 (2,
  '0.099*"vader" + 0.069*"moeder" + 0.040*"vraag" + 0.036*"hoofd" + '
  '0.033*"belle" + 0.031*"matthies" + 0.029*"goed" + 0.020*"groot" + '
  '0.020*"open" + 0.018*"elkaar"'),
 (3,
  '0.100*"obbe" + 0.043*"moeder" + 0.037*"lang" + 0.028*"vader" + '
  '0.027*"groot" + 0.027*"godin" + 0.027*"koud" + 0.024*"dag" + 0.021*"koe" + '
  '0.019*"tijd"'),
 (4,
  '0.076*"vader" + 0.061*"moeder" + 0.034*"jassen" + 0.029*"één" + 0.029*"bil" '
  '+ 0.029*"groot" + 0.027*"lichaam" + 0.024*"nieuw" + 0.019*"belle" + '
  '0.019*"warm"')]


### 3.4. Evaluate LDA model

In [48]:
# Compute Coherence Score (higher is better)
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', round(coherence_lda, 2))

Coherence Score:  0.62


### 3.5. Visualize LDA model

In [50]:
# Enable display of prepared model data
pyLDAvis.enable_notebook()

# Sets the file path
LDAvis_data_filepath = os.path.join('results/ldavis_prepared_Deel_III_'+str(num_topics))

In [51]:
# Save the computed LDA data to file path
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

C:\Users\isabe\anaconda3\envs\tm_master_NL\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [52]:
# Load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
# Save final topic model as html file
pyLDAvis.save_html(LDAvis_prepared, 'results/ldavis_prepared_Deel_III_'+ str(num_topics) +'.html')

In [53]:
# Display the final topic model in a dashboard
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.049771 -0.021852       1        1  22.367594
0      0.083985 -0.100401       2        1  21.326394
3      0.013064  0.027510       3        1  19.863164
4      0.034682  0.121916       4        1  19.459545
1     -0.181502 -0.027174       5        1  16.983303, topic_info=      Term        Freq       Total Category  logprob  loglift
5     obbe   52.000000   52.000000  Default  30.0000  30.0000
93    goed   39.000000   39.000000  Default  29.0000  29.0000
13   vader   90.000000   90.000000  Default  28.0000  28.0000
12  moeder  109.000000  109.000000  Default  27.0000  27.0000
18   vraag   20.000000   20.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
54    tijd    3.241155   14.219306   Topic5  -4.5494   0.2943
5     obbe    4.352025   52.572714   Topic5  -4.2547  -0.7186
85   belle    3.304889   27.444357   Topic5  -4.5300  -0.3438
0    duwen    3.244329   17.528980   Topic5  -4.5485   0.0860
33   keren    3.228060   15.507668   Topic5  -4.5535   0.2035

[239 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
98        1  0.181608  aarde
98        4  0.726430  aarde
25        1  0.320230  alsof
25        2  0.400288  alsof
25        3  0.160115  alsof
...     ...       ...    ...
95        1  0.127401    één
95        2  0.042467    één
95        3  0.084934    één
95        4  0.424670    één
95        5  0.297269    één

[483 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 5, 2])